In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# Function to read multiple CSV files from a directory
def read_csv_files(directory):
    dfs = []
    for filename in os.listdir(directory):
        if filename.endswith('.csv'):
            filepath = os.path.join(directory, filename)
            df = pd.read_csv(filepath)
            dfs.append(df)
    return pd.concat(dfs, ignore_index=True)

In [5]:
# Read all CSV files for races from the "Races" folder
races = read_csv_files(r"C:\Users\neeth\Downloads\Races")

# Read all CSV files for horses from the "Horses" folder
horses = read_csv_files(r"C:\Users\neeth\Downloads\Horses")

# Read the single CSV file for forward data
forward = pd.read_csv(r"C:\Users\neeth\Downloads\forward.csv")

# Display the first few rows of each dataset
print(races.head())
print(horses.head())
print(forward.head())

C:\Users\neeth\AppData\Local\Temp\ipykernel_14868\605732460.py:7: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath)
C:\Users\neeth\AppData\Local\Temp\ipykernel_14868\605732460.py:7: DtypeWarning: Columns (27) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath)


      rid         course   time      date  \
0  271018         Exeter  03:15  90/01/01   
1  275156  Tramore (IRE)  02:00  90/01/01   
2  282203      Catterick  02:45  90/01/01   
3  298761     Cheltenham  02:30  90/01/01   
4  301118        Windsor  03:30  90/01/01   

                                               title rclass   band   ages  \
0  David Garrett Memorial Challenge Trophy Novice...    NaN    NaN  6-8yo   
1              Tattersalls Mares E.B.F. Novice Chase    NaN    NaN    NaN   
2                       Scotch Corner Handicap Chase    NaN  0-125    NaN   
3                           A. S. W. Handicap Hurdle    NaN    NaN    NaN   
4                        Touchen End Handicap Hurdle    NaN  0-115    NaN   

  distance     condition    hurdles                          prizes  \
0     3m1f          Soft  19 fences   [2922.5, 875.0, 420.0, 192.5]   
1       2m          Soft  12 fences                              []   
2    1m7½f  Good To Firm  12 fences          [2238.0,

In [7]:
# Display the first few rows of each dataset
display(races.head())
display(horses.head())
display(forward.head())

,rid,course,time,date,title,rclass,band,ages,distance,condition,hurdles,prizes,winningTime,prize,metric,countryCode,ncond,class,currency
0,271018,Exeter,03:15,90/01/01,David Garrett Memorial Challenge Trophy Novice...,NaN,NaN,6-8yo,3m1f,Soft,19 fences,"[2922.5, 875.0, 420.0, 192.5]",398.3,4409.0,5028.0,GB,5,0,NaN
1,275156,Tramore (IRE),02:00,90/01/01,Tattersalls Mares E.B.F. Novice Chase,NaN,NaN,NaN,2m,Soft,12 fences,[],267.0,NaN,3218.0,IE,5,0,NaN
2,282203,Catterick,02:45,90/01/01,Scotch Corner Handicap Chase,NaN,0-125,NaN,1m7½f,Good To Firm,12 fences,"[2238.0, 618.0, 294.0]",238.0,3150.0,3116.5,GB,2,0,NaN
3,298761,Cheltenham,02:30,90/01/01,A. S. W. Handicap Hurdle,NaN,NaN,NaN,2m,Good To Firm,NaN,"[5117.0, 1526.0, 728.0, 329.0]",243.8,7700.0,3218.0,GB,2,0,NaN
4,301118,Windsor,03:30,90/01/01,Touchen End Handicap Hurdle,NaN,0-115,NaN,2m6f,Good,NaN,"[2344.8, 652.8, 314.4]",330.7,3312.0,4424.0,GB,1,0,NaN


,rid,horseName,age,saddle,decimalPrice,isFav,trainerName,jockeyName,position,positionL,...,OR,father,mother,gfather,runners,margin,weight,res_win,res_place,price
0,271018,Combermere,6.0,0.0,0.222222,0,R G Frost,J Frost,1,NaN,...,NaN,Absalom,Queen's Parade,Sovereign Path,14,1.521003,69,1.0,1.0,NaN
1,271018,Royal Battery,6.0,0.0,0.090909,0,D H Barons,S Earle,2,10,...,NaN,Norfolk Air,All At Sea,Man The Rail,14,1.521003,69,0.0,1.0,NaN
2,271018,Just So,7.0,0.0,0.029412,0,J D Roberts,S Burrough,3,15,...,NaN,Sousa,Just Camilla,Ascertain I,14,1.521003,66,0.0,1.0,NaN
3,271018,Mandraki Shuffle,8.0,0.0,0.090909,0,Oliver Sherwood,M Richards,4,20,...,NaN,Mandalus,Indictment,Desert Call,14,1.521003,69,0.0,0.0,NaN
4,271018,Turnberry Dawn,8.0,0.0,0.047619,0,T B Hallett,P Richards,5,dist,...,NaN,Fair Turn,Shuil Alainn,Levanter,14,1.521003,69,0.0,0.0,NaN


,course,countryCode,marketTime,title,runners,condition,prize,rclass,horseName,trainerName,jockeyName,RPRc,TRc,OR,weightSt,weightLb,age,decimalPrice
0,Limerick,0,2020-09-11 12:45:00+01:00,Irish Stallion Farms EBF Fillies Maiden (Plus ...,14,Yielding To Soft,13717.5,NaN,All Down To Rosie,Conor O'Dwyer,Kevin Manning,NaN,NaN,NaN,9,2,2,50.000000
1,Limerick,0,2020-09-11 12:45:00+01:00,Irish Stallion Farms EBF Fillies Maiden (Plus ...,14,Yielding To Soft,13717.5,NaN,Colfer Kay,K J Condon,W J Lee,79.0,70.0,NaN,9,2,2,6.037778
2,Limerick,0,2020-09-11 12:45:00+01:00,Irish Stallion Farms EBF Fillies Maiden (Plus ...,14,Yielding To Soft,13717.5,NaN,Dha Leath,Garvan Donnelly,J M Sheridan,NaN,NaN,NaN,9,2,2,49.666667
3,Limerick,0,2020-09-11 12:45:00+01:00,Irish Stallion Farms EBF Fillies Maiden (Plus ...,14,Yielding To Soft,13717.5,NaN,Ellabella,Andrew McNamara,Colin Keane,NaN,NaN,NaN,9,2,2,17.944444
4,Limerick,0,2020-09-11 12:45:00+01:00,Irish Stallion Farms EBF Fillies Maiden (Plus ...,14,Yielding To Soft,13717.5,NaN,Fermoy,Mrs John Harrington,Tom Madden,73.0,58.0,NaN,9,2,2,17.594737


In [9]:
# Checking for missing values in races DataFrame
races_missing_values = races.isnull().sum()
print("Missing values in Races DataFrame:")
print(races_missing_values)

Missing values in Races DataFrame:
rid                 0
course              0
time                0
date                0
title               1
rclass         152992
band           231309
ages            13054
distance            0
condition          48
hurdles        274953
prizes              0
winningTime         0
prize           19708
metric              0
countryCode         0
ncond               0
class               0
currency       387291
dtype: int64


In [11]:
# Checking for missing values in horses DataFrame
horses_missing_values = horses.isnull().sum()
print("\nMissing values in Horses DataFrame:")
print(horses_missing_values)


Missing values in Horses DataFrame:
rid                   0
horseName             0
age                 309
saddle           179168
decimalPrice          0
isFav                 0
trainerName        2850
jockeyName          125
position              0
positionL        783662
dist            1074385
weightSt              0
weightLb              0
overWeight      4012988
outHandicap     3994332
headGear        3119695
RPR              864877
TR              1681610
OR              1795263
father             1289
mother             4480
gfather          119374
runners               0
margin                0
weight                0
res_win               0
res_place             0
price           4012304
dtype: int64


In [13]:
# Checking for missing values in forward DataFrame
forward_missing_values = forward.isnull().sum()
print("\nMissing values in Forward DataFrame:")
print(forward_missing_values)


Missing values in Forward DataFrame:
course             11
countryCode         0
marketTime          0
title               0
runners             0
condition           5
prize               0
rclass          10716
horseName           0
trainerName         0
jockeyName         92
RPRc             5597
TRc             10088
OR               9470
weightSt            0
weightLb            0
age                 0
decimalPrice        0
dtype: int64
